https://huggingface.co/Narrativaai/bloom-560m-finetuned-totto-table-to-text

In [18]:
from datasets import load_dataset
from transformers import BloomTokenizerFast, BloomForCausalLM

valid_dataset = load_dataset('totto', split='validation')

from preprocess import preprocess

Found cached dataset totto (/home/mukherjee/.cache/huggingface/datasets/totto/default/1.0.0/263c85871e5451bc892c65ca0306c0629eb7beb161e0eb998f56231562335dd2)


In [19]:
for key, val in valid_dataset[1].items():
    print(key)
    print(val)
    print('\n')

id
1


table_page_title
Alma Jodorowsky


table_webpage_url
http://en.wikipedia.org/wiki/Alma_Jodorowsky


table_section_title
Filmography


table_section_text



table
[[{'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Year'}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Title'}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Role'}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Notes'}], [{'column_span': 1, 'is_header': False, 'row_span': 1, 'value': '2006'}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Gaspard le Bandit'}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Mathilde de Varade'}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Period TV film'}], [{'column_span': 1, 'is_header': False, 'row_span': 3, 'value': '2011'}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Eyes Find Eyes'}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value

In [20]:
valid_dataset = valid_dataset.map(preprocess)

Loading cached processed dataset at /home/mukherjee/.cache/huggingface/datasets/totto/default/1.0.0/263c85871e5451bc892c65ca0306c0629eb7beb161e0eb998f56231562335dd2/cache-6ae62e2c254ef7df.arrow


In [21]:
print(valid_dataset[1]['linearized_table'])

<s><page_title> Alma Jodorowsky </page_title> <section_title> Filmography </section_title> <table> <row> <c> Year </c> <c> Title <row_header> Year </row_header> </c> <c> Role <row_header> Year </row_header> <row_header> Title </row_header> </c> <c> Notes <row_header> Year </row_header> <row_header> Title </row_header> <row_header> Role </row_header> </c> </row> <row> <c> 2006 <col_header> Year </col_header> </c> <c> Gaspard le Bandit <col_header> Title </col_header> </c> <c> Mathilde de Varade <col_header> Role </col_header> </c> <c> Period TV film <col_header> Notes </col_header> </c> </row> <row> <c> 2011 <col_header> Year </col_header> </c> <c> Eyes Find Eyes <col_header> Title </col_header> </c> <c> Estelle <col_header> Role </col_header> </c> <c>  <col_header> Notes </col_header> </c> </row> <row> <c> Section de recherche <col_header> Year </col_header> </c> <c> Caroline <col_header> Title </col_header> </c> <c> Police TV series (1 episode: No.5.3) <col_header> Role </col_header> 

In [22]:
ckpt = "mrm8488/bloom-560m-finetuned-totto-table-to-text"

tokenizer = BloomTokenizerFast.from_pretrained(ckpt)
model = BloomForCausalLM.from_pretrained(ckpt).to("cuda")


def explain_hl_cells(text):
    inputs = tokenizer(text, return_tensors='pt')
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=2048, eos_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(output[0], skip_special_tokens=False)

print(explain_hl_cells(valid_dataset[1]['linearized_table']))

<s><page_title> Alma Jodorowsky </page_title> <section_title> Filmography </section_title> <table> <row> <c> Year </c> <c> Title <row_header> Year </row_header> </c> <c> Role <row_header> Year </row_header> <row_header> Title </row_header> </c> <c> Notes <row_header> Year </row_header> <row_header> Title </row_header> <row_header> Role </row_header> </c> </row> <row> <c> 2006 <col_header> Year </col_header> </c> <c> Gaspard le Bandit <col_header> Title </col_header> </c> <c> Mathilde de Varade <col_header> Role </col_header> </c> <c> Period TV film <col_header> Notes </col_header> </c> </row> <row> <c> 2011 <col_header> Year </col_header> </c> <c> Eyes Find Eyes <col_header> Title </col_header> </c> <c> Estelle <col_header> Role </col_header> </c> <c>  <col_header> Notes </col_header> </c> </row> <row> <c> Section de recherche <col_header> Year </col_header> </c> <c> Caroline <col_header> Title </col_header> </c> <c> Police TV series (1 episode: No.5.3) <col_header> Role </col_header> 

### Debugging

In [23]:
table = valid_dataset[1]['table']

In [24]:
import copy

In [25]:
def _add_adjusted_col_offsets(table):
    """Add adjusted column offsets to take into account multi-column cells."""
    adjusted_table = []
    for row in table:
      real_col_index = 0
      adjusted_row = []
      for cell in row:
        adjusted_cell = copy.deepcopy(cell)
        adjusted_cell["adjusted_col_start"] = real_col_index
        adjusted_cell["adjusted_col_end"] = (
            adjusted_cell["adjusted_col_start"] + adjusted_cell["column_span"])
        real_col_index += adjusted_cell["column_span"]
        adjusted_row.append(adjusted_cell)
      adjusted_table.append(adjusted_row)
    return adjusted_table

In [26]:
adjusted_table = _add_adjusted_col_offsets(table)
print(adjusted_table)

[[{'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Year', 'adjusted_col_start': 0, 'adjusted_col_end': 1}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Title', 'adjusted_col_start': 1, 'adjusted_col_end': 2}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Role', 'adjusted_col_start': 2, 'adjusted_col_end': 3}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Notes', 'adjusted_col_start': 3, 'adjusted_col_end': 4}], [{'column_span': 1, 'is_header': False, 'row_span': 1, 'value': '2006', 'adjusted_col_start': 0, 'adjusted_col_end': 1}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Gaspard le Bandit', 'adjusted_col_start': 1, 'adjusted_col_end': 2}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Mathilde de Varade', 'adjusted_col_start': 2, 'adjusted_col_end': 3}, {'column_span': 1, 'is_header': False, 'row_span': 1, 'value': 'Period TV film', 'adjusted_col_start': 3, 'adjusted_col_end': 4}], 

In [27]:
def _get_heuristic_row_headers(adjusted_table, row_index, col_index):
    """Heuristic to find row headers."""
    row_headers = []
    row = adjusted_table[row_index]
    for i in range(0, col_index):
      if row[i]["is_header"]:
        row_headers.append(row[i])
    return row_headers

In [28]:
for r_index, row in enumerate(table):
    print(row)
    print('\n')
    row_headers = _get_heuristic_row_headers(adjusted_table, r_index, c_index)
    print(row_headers)
    break

[{'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Year'}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Title'}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Role'}, {'column_span': 1, 'is_header': True, 'row_span': 1, 'value': 'Notes'}]


[]


In [29]:
def _get_heuristic_col_headers(adjusted_table, row_index, col_index):
    """Heuristic to find column headers."""
    adjusted_cell = adjusted_table[row_index][col_index]
    adjusted_col_start = adjusted_cell["adjusted_col_start"]
    adjusted_col_end = adjusted_cell["adjusted_col_end"]
    col_headers = []
    for r in range(0, row_index):
      row = adjusted_table[r]
      for cell in row:
        if (cell["adjusted_col_start"] < adjusted_col_end and
            cell["adjusted_col_end"] > adjusted_col_start):
          if cell["is_header"]:
            col_headers.append(cell)

    return col_headers

In [30]:
subject = valid_dataset[1]['table_page_title']
# print(subject)
cell_indices = valid_dataset[1]["highlighted_cells"]
all_tripples = list()
highlighted_tripples = list()

table_str = "<table> "
highlight_str = "<highlight>"

for r_index, row in enumerate(table):
    row_str = "<row> "
    highlight_row_str = "<row> "
    
    for c_index, col in enumerate(row):
        start_cell_marker = "<c> "
        end_cell_marker = "</c> "
        col_headers = _get_heuristic_col_headers(adjusted_table, r_index, c_index)
        if col_headers:
            # print(col)
            # print(col_headers)
            # print('\n')
            
            all_tripples.append((subject, col_headers[0]['value'], col['value']))
            
            item_str = start_cell_marker + col["value"] + " "

            for col_header in col_headers:
                item_str += "<col_header> " + col_header["value"] + " </col_header> "

            for row_header in row_headers:
                item_str += "<row_header> " + row_header["value"] + " </row_header> "

            item_str += end_cell_marker
            row_str += item_str
            
            if [r_index, c_index] in cell_indices:
                highlighted_tripples.append((subject, col_headers[0]['value'], col['value']))
                highlight_row_str += item_str

    row_str += "</row> "
    table_str += row_str
    
    
    highlight_row_str += "</row> "
    highlight_str += highlight_row_str
    
table_str += "</table>"
highlight_str += "</highlight>"

In [32]:
print(all_tripples[:10])

[('Alma Jodorowsky', 'Year', '2006'), ('Alma Jodorowsky', 'Title', 'Gaspard le Bandit'), ('Alma Jodorowsky', 'Role', 'Mathilde de Varade'), ('Alma Jodorowsky', 'Notes', 'Period TV film'), ('Alma Jodorowsky', 'Year', '2011'), ('Alma Jodorowsky', 'Title', 'Eyes Find Eyes'), ('Alma Jodorowsky', 'Role', 'Estelle'), ('Alma Jodorowsky', 'Notes', ''), ('Alma Jodorowsky', 'Year', 'Section de recherche'), ('Alma Jodorowsky', 'Title', 'Caroline')]


In [33]:
print(highlighted_tripples)

[('Alma Jodorowsky', 'Year', '2016'), ('Alma Jodorowsky', 'Title', 'Kids in Love'), ('Alma Jodorowsky', 'Role', 'Evelyn')]


In [34]:
print(table_str)

<table> <row> </row> <row> <c> 2006 <col_header> Year </col_header> </c> <c> Gaspard le Bandit <col_header> Title </col_header> </c> <c> Mathilde de Varade <col_header> Role </col_header> </c> <c> Period TV film <col_header> Notes </col_header> </c> </row> <row> <c> 2011 <col_header> Year </col_header> </c> <c> Eyes Find Eyes <col_header> Title </col_header> </c> <c> Estelle <col_header> Role </col_header> </c> <c>  <col_header> Notes </col_header> </c> </row> <row> <c> Section de recherche <col_header> Year </col_header> </c> <c> Caroline <col_header> Title </col_header> </c> <c> Police TV series (1 episode: No.5.3) <col_header> Role </col_header> </c> </row> <row> <c> Awake to Emptiness <col_header> Year </col_header> </c> <c> Alma <col_header> Title </col_header> </c> <c> Short film <col_header> Role </col_header> </c> </row> <row> <c> 2012 <col_header> Year </col_header> </c> <c> Sea, No Sex and Sun <col_header> Title </col_header> </c> <c> Diane <col_header> Role </col_header> </c

In [35]:
print(highlight_str)

<highlight><row> </row> <row> </row> <row> </row> <row> </row> <row> </row> <row> </row> <row> </row> <row> </row> <row> <c> 2016 <col_header> Year </col_header> </c> <c> Kids in Love <col_header> Title </col_header> </c> <c> Evelyn <col_header> Role </col_header> </c> </row> <row> </row> <row> </row> </highlight>


In [36]:
from nltk.util import ngrams
import re

from nltk.corpus import stopwords
stop = stopwords.words('english')

def trigrams_text(s):
    s = s.lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != "" and token not in (stop)]
    output = list(ngrams(tokens, 3))
    return output

In [37]:
desc = valid_dataset[1]['sentence_annotations']['final_sentence'][0]
print(desc)
print(trigrams_text(desc))

Alma Jodorowsky had the role of Evelyn in 2016 film Kids in Love.
[('alma', 'jodorowsky', 'role'), ('jodorowsky', 'role', 'evelyn'), ('role', 'evelyn', '2016'), ('evelyn', '2016', 'film'), ('2016', 'film', 'kids'), ('film', 'kids', 'love')]
